# DATA PREPARATION _ MAIN 

In [1]:
# spark.stop()

In [2]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when - is removed, remove also this line and adapt imports
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
    
    
import pykhaos.utils.notebooks as nb
   
RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    print("Running from notebook")
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    
if not RUNNING_FROM_NOTEBOOK:
    args = my_project.arg_parser()

20181005-121526 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20181005_121526.log


Running from notebook


In [3]:
import time
from churn.utils.constants import *
import pykhaos.utils.pyspark_configuration as pyspark_config

def __init_spark():

    start_time = time.time()
    app_name = APP_NAME if APP_NAME else os.environ.get('USER', '') \
                                         + "_" + PROJECT_NAME \
                                         + "_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S")

    sc, spark, sql_context = pyspark_config.get_spark_session(app_name=app_name, log_level="OFF")
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))

    return spark

In [4]:
spark = __init_spark()

Ended spark session: 1087.10340405 secs | default parallelism=2


In [3]:
import yaml


default_filename = os.path.join(USECASES_SRC, "churn", "input", "config.yaml")

input_file = default_filename if RUNNING_FROM_NOTEBOOK else input_filename


config = yaml.load(open(input_file))

import pprint
pprint.pprint(config)




{'closing_day': 20180831,
 'cycles_horizon': 4,
 'discarded_cycles': 0,
 'level': 'service',
 'model_target': 'port',
 'segment_filter': 'allmob',
 'service_set': 'movil',
 'sources': {'ids': {'address': False,
                     'billing': False,
                     'call_centre_calls': False,
                     'campaigns': False,
                     'customer': True,
                     'customer_penalties': False,
                     'device_catalogue': False,
                     'geneva_traffic': False,
                     'netscout': False,
                     'orders': False,
                     'services': True,
                     'tnps': False}}}


In [28]:
# closing_day = config["closing_day"]
# segment_filter = config["segment_filter"] # segmento al que nos dirigimos
# level = config["level"] # service=(msisdn), num_client, nif 
# service_set = config["service_set"] # siempre movil
# model_target = config["model_target"] 
# cycles_horizon=config["cycles_horizon"]
# discarded_cycles=config["discarded_cycles"]

In [63]:
from churn.datapreparation.churn_data_loader import ChurnDataLoader

if cycles_horizon<=0:
    print("[ERROR] cycles horizon must be a number greater than 0 (inserted {})".format(cycles_horizon))
    import sys
    sys.exit(1)
    

cdl = ChurnDataLoader(spark, config)


df_labeled_car = cdl.getLabeledCar(feats=None,target=None)


Calling get_numclients_under_analysis for closing_day=20180814 and segment_filter=onlymob
('df_customer', 5623176)
('df_service', 11795811)
('df_services', 10677111)
('get_numclients_under_analysis - df_customer_aggregations', 5318051)
[Info DataLoader] Number of target services before labeling:942118 
closing day = 20180814
discarded period: 20180815-20180920
target period: 20180921-20181021
end reading source customer
end reading source services
Ended process of reading ids sources
('df_target_num_clients', 'num_cliente')
('after df_car', 'TRATAMIENTO,NOMBRE,PRIM_APELLIDO,SEG_APELLIDO,CLASE_CLI_COD_CLASE_CLIENTE,DIR_LINEA1,DIR_LINEA2,DIR_LINEA3,COD_ESTADO_GENERAL,NOM_COMPLETO,DIR_FACTURA1,DIR_FACTURA2,DIR_FACTURA3,DIR_FACTURA4,CODIGO_POSTAL,TRAT_FACT,NOMBRE_CLI_FACT,APELLIDO1_CLI_FACT,APELLIDO2_CLI_FACT,DIR_NUM_DIRECCION,NIF_CLIENTE,FECHA_NACI,METODO_PAGO,PUBLICIDAD,ENCUESTAS,CTA_CORREO_CONTACTO,CTA_CORREO,FACTURA_CATALAN,FACTURA_ELECTRONICA,SUPEROFERTA,NIF_FACTURACION,TIPO_DOCUMENTO

In [7]:
from churn.datapreparation.data_preparation_main import save_df

dir_name = "/user/csanc109/projects/churn/port/allmob/"
ret_stat = create_directory(os.path.join(dir_name))


Built filename: /user/csanc109/projects/churn/port/allmob/df_port_allmob_service_20180831_c4_d0
Created directory returned False


AttributeError: 'str' object has no attribute 'write'

In [13]:
from pykhaos.utils.date_functions import move_date_n_cycles

end_date = move_date_n_cycles("20180831", -4)

end_date

(0, '20180821')
(1, '20180814')
(2, '20180807')
(3, '20180731')


'20180731'